## Langchain and Hybrid Search using Pinecone

In [8]:
api_key ="e1398260-7e35-43d4-a3b4-7b5a3f2ce7ca"

In [6]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks


In [5]:
from langchain_community.retrievers import PineconeHybridSearchRetriever


In [3]:
%pip install langchain_community --upgrade

     ---------------------------------------- 0.0/51.6 kB ? eta -:--:--
     ---------------------------------------  51.2/51.6 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 51.6/51.6 kB 881.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.2 MB 8.6 MB/s eta 0:00:01
   ---------- ----------------------------- 0.6/2.2 MB 6.1 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.2 MB 6.8 MB/s eta 0:00:01
   ----------------- ---------------------- 1.0/2.2 MB 6.8 MB/s eta 0:00:01
   -------------------- ------------------- 1.2/2.2 MB 5.2 MB/s eta 0:00:01
   -------------------- ------------------- 1.2/2.2 MB 5.2 MB/s eta 0:00:01
   ---------------------- ----------------- 1.2/2.2 MB 3.9 MB/s eta 0:00:01
   ----------------------- ---------------- 1.3/2.2 MB 3.5 MB/s eta 0:00:01
   ------------------------- -------------- 1.5/2.2 MB 3.6 MB/s eta 0:00:01
   ------------

In [13]:
import os
from pinecone import Pinecone, ServerlessSpec

api_key = "e1398260-7e35-43d4-a3b4-7b5a3f2ce7ca"  # Replace with your actual Pinecone API key
index_name = "hybrid-search-langchain-pinecone"

# Create an instance of the Pinecone client
pc = Pinecone(api_key=api_key)

# Check if the index exists, and create it if it doesn't
if index_name not in pc.list_indexes():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric='dotproduct',
        spec=ServerlessSpec(cloud='aws', region="us-east-1")
    )


In [14]:
index=pc.Index(index_name)
index

In [15]:
!pip install langchain_huggingface


   ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
   ---------------------------------------- 227.1/227.1 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
embeddings


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [20]:
from pinecone_text.sparse import BM25Encoder
bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt to C:\Users\Navdeep
[nltk_data]     walia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\Navdeep
[nltk_data]     walia\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [24]:
sentences=["During my trip to Japan, I was mesmerized by the serene beauty of Kyoto's ancient temples and stunning gardens.",
"Exploring the vibrant streets of New York City, I enjoyed the hustle and bustle of Times Square and the breathtaking views from the top of the Empire State Building.",
"A visit to the picturesque Amalfi Coast in Italy left me in awe of the charming seaside villages, dramatic cliffs, and crystal-clear waters."]

bm25_encoder.fit(sentences)

bm25_encoder.dump("bm25_values.json")

bm25_encoder=BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 1857.26it/s]


In [25]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index) 

In [26]:
retriever.add_texts(["During my trip to Japan, I was mesmerized by the serene beauty of Kyoto's ancient temples and stunning gardens.",
"Exploring the vibrant streets of New York City, I enjoyed the hustle and bustle of Times Square and the breathtaking views from the top of the Empire State Building.",
"A visit to the picturesque Amalfi Coast in Italy left me in awe of the charming seaside villages, dramatic cliffs, and crystal-clear waters."]
)

100%|██████████| 1/1 [00:03<00:00,  3.72s/it]


In [27]:
retriever.invoke("What city did i visit last?")

[Document(page_content='A visit to the picturesque Amalfi Coast in Italy left me in awe of the charming seaside villages, dramatic cliffs, and crystal-clear waters.'),
 Document(page_content='Exploring the vibrant streets of New York City, I enjoyed the hustle and bustle of Times Square and the breathtaking views from the top of the Empire State Building.'),
 Document(page_content="During my trip to Japan, I was mesmerized by the serene beauty of Kyoto's ancient temples and stunning gardens.")]